In [1]:
import sys
import time
import json
import os
#sys.path.insert(0, 'src/data')
#sys.path.insert(0, 'src/dgl_graphsage')

import recommend as r
from train_updated import train
from api.spotifyAPI import SpotifyAPI
from utils import load_graph
from utils import load_features
from sklearn.neighbors import NearestNeighbors
import pandas as pd
from dgl import load_graphs
import matplotlib.pyplot as plt
import networkx as nx
import dgl


Using backend: pytorch


In [2]:
%%time
# Loading Graph ~ 3min
feat_dir = "../../data/a13group1/features/merged_features.csv"
double_edge_dir = "../../data/a13group1/double_edges_170k.bin"
feat_data, uri_map = load_features(feat_dir)
dgl_G = nx.read_gpickle("../../data/a13group1/graph_170k.gpickle")
dgl_G, weights = load_graph(dgl_G, uri_map)

CPU times: user 1min 57s, sys: 21.3 s, total: 2min 19s
Wall time: 2min 17s


In [3]:
#dgl_G_cuda = dgl_G.to('cuda')
#weights = weights.to('cuda')
#feat_data = feat_data.to('cuda')

In [4]:
%%time
# Training the Model. GPU ~ 00:00:40
with open('../../config/model-params.json') as fh:
            model_cfg = json.load(fh)
model, pred, losses = train(dgl_G, weights.to('cuda'), feat_data, cuda=True, feat_dim=14, emb_dim=10, test_data=False)

# Put everything on CPU
model = model.to('cpu')
pred = pred.to('cpu')

Train pos edge: 26413580
Validation pos edge: 518464
Cuda enabled: True

Training starts:
In epoch 1 batch 1, loss: 0.7261016368865967


KeyboardInterrupt: 

In [4]:
# Create Embeddings
z = model(dgl_G, feat_data)

# Create Predictions
# In the form of the strength of the connection between source, destination from dgl_G.edges()
preds = pred(dgl_G, z)

In [8]:
# Create Nearest Neighbors
neigh = NearestNeighbors(n_neighbors=25, radius=0.4)
neigh.fit(feat_data)

NearestNeighbors(n_neighbors=25, radius=0.4)

In [9]:
# Get a random playlist
item = r.get_random_playlist()

/teams/DSC180A_FA21_A00/a13group1/data/mpd.slice.7000-7999.json


In [10]:
# Get playlist information
item, original_tracks, seeds = r.get_playlist_info(item)
original_tracks

Playlist ID: 7009
Playlist Length: 20


['Sneakin’---Drake',
 'Gassed Up---Nebu Kiniza',
 'You Was Right---Lil Uzi Vert',
 'Fire Squad - Live---J. Cole',
 'A Tale of 2 Citiez - Live---J. Cole',
 'Deja Vu---J. Cole',
 '20 Joints---Berner',
 'Moves---Big Sean',
 'Look At Me!---XXXTENTACION',
 'Rent Money---Future',
 'No Heart---21 Savage',
 'Dear Mama---2Pac',
 'Neighbors---J. Cole',
 '4 Your Eyez Only---J. Cole',
 'Miss America---J. Cole',
 'Like Toy Soldiers---Eminem',
 'T-Shirt---Migos',
 'Get Your Walk On---Xzibit',
 'X---Xzibit',
 'I Need A Doctor---Dr. Dre']

In [11]:
# Get recommendations
uri_recs = r.recommend(seeds, dgl_G, z, pred, neigh, feat_data, uri_map)
uri_recs

['47jNy3vE0aBM8nWWFAZaX2',
 '47jNy3vE0aBM8nWWFAZaX2',
 '5XlpERwWF8SuVUL9AghEIh',
 '6jalstnZd8hEJuLGFFjL9L',
 '5rSH2YlI18D4mmgP0tQbs9',
 '7dEYcnW1YSBpiKofefCFCf',
 '1eLU27PPFxBhbp1CBjPh9J',
 '3hxsFtkwEx6rZ310ROjwPD',
 '08azaGQ1YKWvajMDARPSeZ',
 '7dEYcnW1YSBpiKofefCFCf',
 '63Zqn50wQCs8G1fci05C8a',
 '0pSaDwrMLM7F82LWJg9v6s',
 '20WsdoJsGsjVOLtA0GB5hL',
 '5XlpERwWF8SuVUL9AghEIh',
 '7bFElqzAlHXnnINRBXRQbK',
 '7mcPcohfR96oIGbv6dxS3Q',
 '5XlpERwWF8SuVUL9AghEIh',
 '71u5SjnkSQgzIt1UzHchbi',
 '7crrp07AQujRvbOoIGitk8',
 '6B0FIZm6Xs4qYRyTjsuU0Y']

In [12]:
# Translate to Song Names
client_id = ''
client_secret = ''

spotify = SpotifyAPI(client_id, client_secret)

In [13]:
%%time
rec_track_names = r.get_rec_names(uri_recs, spotify, 2)

Exception: Could not authenticate client.

In [14]:
original_tracks

['Sneakin’---Drake',
 'Gassed Up---Nebu Kiniza',
 'You Was Right---Lil Uzi Vert',
 'Fire Squad - Live---J. Cole',
 'A Tale of 2 Citiez - Live---J. Cole',
 'Deja Vu---J. Cole',
 '20 Joints---Berner',
 'Moves---Big Sean',
 'Look At Me!---XXXTENTACION',
 'Rent Money---Future',
 'No Heart---21 Savage',
 'Dear Mama---2Pac',
 'Neighbors---J. Cole',
 '4 Your Eyez Only---J. Cole',
 'Miss America---J. Cole',
 'Like Toy Soldiers---Eminem',
 'T-Shirt---Migos',
 'Get Your Walk On---Xzibit',
 'X---Xzibit',
 'I Need A Doctor---Dr. Dre']

In [18]:
pd.Series(rec_track_names).value_counts()

Got It (feat. Snoop Dogg)---Boys Noize                                                                           3
Bad Meets Evil---Eminem                                                                                          2
Ka-Bang! (feat. MF Doom)---CZARFACE                                                                              2
Just Life---Mark Battles                                                                                         1
Inertia---ARTY                                                                                                   1
Love Yourz - Live---J. Cole                                                                                      1
1Train (feat. Kendrick Lamar, Joey Bada$$, Yelawolf, Danny Brown, Action Bronson & Big K.R.I.T.)---A$AP Rocky    1
Rap Up 2016---Uncle Murda                                                                                        1
Do Your Time---Ludacris                                                         

Note: We kept repeats because a song that is recommended as the best more than once throughout the playlist should be ranked higher